In [29]:
import numpy as np
import pandas as pd
import sklearn.model_selection as skl_model_selection
import cv2


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, Conv2DTranspose, Conv2D, Input, BatchNormalization, MaxPooling2D, concatenate
from keras.optimizers import Adam
from keras.saving import load_model
from keras.utils import to_categorical
from keras.utils import Sequence
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import kagglehub
import os

import matplotlib.pyplot as plt

In [30]:
path = kagglehub.dataset_download("tawsifurrahman/tuberculosis-tb-chest-xray-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/tawsifurrahman/tuberculosis-tb-chest-xray-dataset/versions/3


In [31]:
#Vamos a intentar obtener las imagenes, para eso primero debemos obtener la tabla

base_path = "TB_Chest_Radiography_Database"
tuberculosis_metadata = pd.read_excel(os.path.join(path, base_path,"Tuberculosis.metadata.xlsx"))
normal_metadata = pd.read_excel(os.path.join(path, base_path,"Normal.metadata.xlsx"))


dataset_tuberculosis_ids = tuberculosis_metadata["FILE NAME"]
dataset_normal_ids = normal_metadata["FILE NAME"]


dataset_tuberculosis_ids = np.array(dataset_tuberculosis_ids)
dataset_normal_ids = np.array(dataset_normal_ids)

dataset_tuberculosis_ids = "Tuberculosis/" + dataset_tuberculosis_ids
dataset_normal_ids = "Normal/" + dataset_normal_ids

val_dataset = np.concatenate((dataset_tuberculosis_ids[:50], dataset_normal_ids[:50]))
val_target = np.concatenate((np.ones(50), np.zeros(50)))


In [45]:
class ImageMaskGenerator(Sequence):
  def __init__(self, dataset_ids, dataset_label, dataset_path, batch_size=32, target_size=(224, 224), **kwargs):
    super().__init__(**kwargs)
    self.dataset_ids = dataset_ids
    self.dataset_label = dataset_label
    self.dataset_path = dataset_path
    self.batch_size = batch_size
    self.target_size = target_size
    self.indexes = np.arange(len(self.dataset_ids))

  def __len__(self):
    return int(np.floor(len(self.dataset_ids) / self.batch_size))

  def __getitem__(self, index):
    batch_ids = self.dataset_ids[index * self.batch_size: (index + 1) * self.batch_size]
    images = []
    masks = []
    labels = []
    for id in batch_ids:
      img_path = f"{self.dataset_path}/{id}.png"

      img = load_img(path=img_path, color_mode="grayscale", target_size=self.target_size)

      img_array = img_to_array(img)

      img_array /=255
      mask_array = np.zeros_like(img_array)

      label = self.get_classification_label(id)

      images.append(img_array)
      masks.append(mask_array)
      labels.append(label)
    return np.array(images), {"segmentation": np.array(masks), "classification": np.array(labels)}

  def get_classification_label(self, id):
    ind = np.where(self.dataset_ids == id)[0][0]
    return self.dataset_label[ind]

  def on_epoch_end(self):
    np.random.shuffle(self.indexes)


In [46]:
batch_size = 10
train_generator = ImageMaskGenerator(val_dataset, val_target, os.path.join(path, base_path), batch_size=batch_size)

In [47]:
model = load_model("./model-it-04.h5")

model.compile(
    optimizer='adam',
    loss={
        "segmentation": "binary_crossentropy",
        "classification": "binary_crossentropy",
    },
    metrics={
        "segmentation": "accuracy",
        "classification": "accuracy",
    }
)

evaluation = model.evaluate(train_generator, return_dict=True)

10/10 ━━━━━━━━━━━━━━━━━━━━ 126s 12s/step - classification_accuracy: 0.4868 - classification_loss: 3.7315 - loss: 5.1757 - segmentation_accuracy: 0.7944 - segmentation_loss: 1.4441
{'classification_accuracy': 0.44999998807907104, 'classification_loss': 4.068774223327637, 'loss': 5.777108192443848, 'segmentation_accuracy': 0.7859352827072144, 'segmentation_loss': 1.708333969116211}


In [49]:
accuracy = evaluation["classification_accuracy"]
print(f"La presición del primer modelo para la clasificación es de {accuracy}")

La presición del modelo para la clasificación es de 0.44999998807907104


In [53]:
model = load_model("./modelv2-01.keras")

model.compile(
    optimizer='adam',
    loss={
        "segmentation": "binary_crossentropy",
        "classification": "binary_crossentropy",
    },
    metrics={
        "segmentation": "accuracy",
        "classification": "accuracy",
    }
)

evaluation = model.evaluate(train_generator, return_dict=True)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 74 variables whereas the saved optimizer has 146 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


10/10 ━━━━━━━━━━━━━━━━━━━━ 126s 12s/step - classification_accuracy: 0.1837 - classification_loss: 3.5403 - loss: 6.8384 - segmentation_accuracy: 0.5539 - segmentation_loss: 3.2981
{'classification_accuracy': 0.4000000059604645, 'classification_loss': 2.5450263023376465, 'loss': 5.380401134490967, 'segmentation_accuracy': 0.6006415486335754, 'segmentation_loss': 2.8353734016418457}


In [54]:
accuracy = evaluation["classification_accuracy"]
print(f"La presición del primer modelo para la clasificación es de {accuracy}")

La presición del primer modelo para la clasificación es de 0.4000000059604645
